# Training and Deploying a PyTorch model with Amazon SageMaker Local Mode

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
!pip install 'sagemaker[local]' --upgrade

In [ ]:
!sudo service docker restart

In [ ]:
!docker rmi -f $(docker images -a -q)

In [ ]:
s3_bucket = 'sagemaker-cookbook-bucket'
prefix = 'chapter03'

In [ ]:
train_s3 = \
f"s3://{s3_bucket}/{prefix}/synthetic/training_data.csv" 

In [ ]:
from sagemaker.inputs import TrainingInput
    
train_input = TrainingInput(train_s3, content_type="text/csv")

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.local import LocalSession

sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}
role = get_execution_role()

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='pytorch_training.py',
                    session=sagemaker_session,
                    role=role,
                    instance_count=1,
                    instance_type='local',
                    framework_version='1.5.0',
                    py_version='py3')

In [ ]:
estimator.fit({'train': train_input})

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(model_data=estimator.model_data, 
                             role=role, 
                             entry_point='pytorch_inference.py', 
                             framework_version='1.5.0',
                             py_version="py3")

In [ ]:
predictor = pytorch_model.deploy(instance_type='local', 
                                 initial_instance_count=1)

In [ ]:
import numpy as np

predictor.predict(np.array([[100], [200]], dtype=np.float32))

In [ ]:
!mkdir -p tmp

In [ ]:
all_s3 = f"s3://{s3_bucket}/{prefix}/synthetic/all_data.csv" 
!aws s3 cp {all_s3} tmp/all_data.csv

In [ ]:
import pandas as pd

all_data = pd.read_csv("tmp/all_data.csv", header=None)
x = all_data[[1]].values
y = all_data[[0]].values

In [ ]:
from numpy import arange

line_x = arange(-5000, 5000, 10)
line_x

In [ ]:
input_data = np.array(line_x.reshape(-1, 1), dtype=np.float32)
result = predictor.predict(input_data)

result

In [ ]:
line_y = result

In [ ]:
from matplotlib import pyplot
pyplot.plot(line_x, line_y, 'r')
pyplot.scatter(x,y,s=1)
pyplot.show()

In [ ]:
predictor.delete_endpoint()